In [13]:
import pandas as pd
from torch.utils.data import Dataset, Subset, DataLoader
# from torchvision import datasets, models, transforms
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
# import math
import time
from pathlib import Path

torch.manual_seed(42)

num_epochs = 30  # each epoch is one pass over the whole dataset
batch_size = 10  # how many samples per training step (32,64,128)
num_workers = 7  # how many CPU cores to use

n_features = 1000 # How many features per image


# Specify some file names
img_features_file = "data/ing_feature_counts.csv"
train_triplets_file = "train_triplets.txt" #"train_triplets.txt"
test_triplets_file = "test_triplets.txt" #"test_triplets.txt"


In [23]:
class FoodTriplets(Dataset):
    """
    Class to load food triplets. Individual items consist of:
     - x: concatenated image features of a triplet (ABC)
     - y: corresponding label (1: B is more similar, 0: C is more similar, None: no label)
    """
    def __init__(self, features_file, triplets_file, is_labelled_data=False):
        print("initializing " + str(triplets_file) + " dataset...")
        
#         self.img_features = pd.read_csv("data/" + features_file, header=None, index_col=0)
        # First row is header labels
        self.img_features = pd.read_csv(img_features_file, index_col=0, header=0)
 
        self.triplets = pd.read_csv("data/" + triplets_file, sep=" ", header=None).to_numpy()
        self.is_labelled_data = is_labelled_data
        self.labels = None

        if self.is_labelled_data:
            # For each triplet ABC add a triplet ACB
            # Also add corresponding labels
            dim = self.triplets.shape
            extended_triplets = np.zeros((2*dim[0], dim[1]), dtype=np.int32)
            self.labels = np.zeros(2*dim[0], dtype=bool)
            idx = 0
            for triplet in self.triplets:
                extended_triplets[idx, :] = triplet
                extended_triplets[idx + 1, :] = [triplet[0], triplet[2], triplet[1]]
                self.labels[idx] = 1  # label at idx+1 is already initialized to 0
                idx += 2

            self.triplets = extended_triplets

        print("done")

    def __len__(self):
        return len(self.triplets)

    def __getitem__(self, idx):
        triplet = self.triplets[idx]
        a, b, c = triplet[0], triplet[1], triplet[2]
        a_features = self.img_features.loc[[a]].to_numpy(dtype=np.float32)
        b_features = self.img_features.loc[[b]].to_numpy(dtype=np.float32)
        c_features = self.img_features.loc[[c]].to_numpy(dtype=np.float32)
        features = np.squeeze(np.concatenate((a_features, b_features, c_features), axis=1))
        label = 0  # dummy label
        if self.is_labelled_data:
            label = np.array([self.labels[idx]])
        return {"x": features, "y": label}

def get_model(device):
    return SimpleNetwork().to(device)

class SimpleNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        n_features = 669

        nin = 3*n_features
        n1 = 2*n_features
        n2 = n_features
        n3 = int(n_features/2)
        n4 = int(n_features/4)
        n5 = int(n_features/8)
        nout = 1

        p_dropout1 = 0.3
        p_dropout2 = 0.5
        p_dropout3 = 0.5
        p_dropout4 = 0.5
        p_dropout5 = 0.2

        self.classifier = nn.Sequential(
            nn.Linear(in_features=669, out_features=333),
            nn.BatchNorm1d(333),
            nn.ReLU(inplace=True),
            nn.Dropout(p=p_dropout1, inplace=False),
            nn.Linear(in_features=333, out_features=165),
            nn.BatchNorm1d(165),
            nn.ReLU(inplace=True),
#             nn.Dropout(p=p_dropout2, inplace=False),
            nn.Linear(in_features=165, out_features=80),
            nn.BatchNorm1d(80),
            nn.ReLU(inplace=True),
#             nn.Dropout(p=p_dropout3, inplace=False),
            nn.Linear(in_features=80, out_features=40),
            nn.BatchNorm1d(40),
            nn.ReLU(inplace=True),
#             nn.Dropout(p=p_dropout4, inplace=False),
            nn.Linear(in_features=40, out_features=1),
#             nn.ReLU(inplace=True),
#             nn.BatchNorm1d(125),
# #             nn.Dropout(p=p_dropout5, inplace=False),
#             nn.Linear(in_features=125, out_features=1),
            nn.Sigmoid()
        )
        
#         self.classifier = nn.Sequential(
#             nn.Linear(in_features=nin, out_features=n1),
#             nn.BatchNorm1d(n1),
#             nn.ReLU(inplace=True),
#             nn.Dropout(p=p_dropout1, inplace=False),
#             nn.Linear(in_features=n1, out_features=n2),
#             nn.BatchNorm1d(n2),
#             nn.ReLU(inplace=True),
#             nn.Dropout(p=p_dropout2, inplace=False),
#             nn.Linear(in_features=n2, out_features=n3),
#             nn.BatchNorm1d(n3),
#             nn.ReLU(inplace=True),
#             nn.Dropout(p=p_dropout3, inplace=False),
#             nn.Linear(in_features=n3, out_features=n4),
#             nn.BatchNorm1d(n4),
#             nn.ReLU(inplace=True),
#             nn.Dropout(p=p_dropout4, inplace=False),
#             nn.Linear(in_features=n4, out_features=n5),
#             nn.ReLU(inplace=True),
#             nn.BatchNorm1d(n5),
#             nn.Dropout(p=p_dropout5, inplace=False),
#             nn.Linear(in_features=n5, out_features=nout),
#             nn.Sigmoid()
#         )


    def forward(self, x):
        logits = self.classifier(x)
        return logits

In [14]:
# initialize datasets
train_data = FoodTriplets(img_features_file, train_triplets_file, is_labelled_data=True)
test_data = FoodTriplets(img_features_file, test_triplets_file, is_labelled_data=False)

initializing train_triplets.txt dataset...
done
initializing test_triplets.txt dataset...
done


In [21]:
# Split train set into train and test set to assess accuracy on unused set
l_train = len(train_data)
val_size = int(0.8 * l_train + 1)
indices = list(range(l_train))
np.random.shuffle(indices)
val_indices, t_indices = indices[:val_size], indices[val_size:]


train_trainloader = DataLoader(torch.utils.data.Subset(train_data, t_indices), batch_size=batch_size, shuffle=True, num_workers=num_workers)
train_testloader = DataLoader(torch.utils.data.Subset(train_data, val_indices), batch_size=batch_size, shuffle=True, num_workers=num_workers)

test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False, num_workers=num_workers)

for i, data, in enumerate(train_trainloader, 1):
    print(data['x'].shape)
    break
    


torch.Size([10, 669])


In [16]:
def experiment(num_epochs, trainloader, device, model, optimizer, scheduler):
    train_results = {}
    test_results = {}
    # Initial test error
    loss, acc, time = test(device, model)
    print(f'Upon initialization. [Test] \t Time {time.avg:.2f} \
            Loss {loss.avg:.2f} \t Accuracy {acc.avg:.2f}')
    test_results[0] = (loss, acc, time)


    for epoch in range(1, num_epochs+1):
        loss, acc, time = train(trainloader, device, model, optimizer, scheduler)
        print(f'Epoch {epoch}. [Train] \t Time {time.sum:.2f} Loss \
                {loss.avg:.2f} \t Accuracy {acc.avg:.2f}')
        train_results[epoch] = (loss.avg, acc.avg, time.avg)

#         if not (epoch % 2):
        if True:
          loss, acc, time = test(device, model)
          print(f'Epoch {epoch}. [Test] \t Time {time.sum:.2f} T-Loss \
                {loss.avg:.2f} \t T-Accuracy {acc.avg:.2f}')
          test_results[epoch] = (loss.avg, acc.avg, time.avg)

def train(trainloader, device, model, optimizer, scheduler):
    time_ = AverageMeter()
    loss_ = AverageMeter()
    acc_ = AverageMeter()
    model.train()

    for i, data, in enumerate(trainloader, 1):
        # Accounting
        end = time.time()

        # get the inputs; data is a list of [inputs, labels]
        inputs = data["x"].float()
        labels = data["y"].float()
        inputs = inputs.to(device)
        labels = labels.to(device)
        bs = inputs.size(0)
        # zero the parameter gradients
        optimizer.zero_grad()  # all the tensors have .grad attribute
        # forward propagation
        logits = model(inputs) # forward propagation
        loss = criterion(logits, labels) # computing the loss for predictions
        # Backward propagation
        loss.backward() # backpropgation
        # Optimization step.
        optimizer.step() # applying an optimization step

        # Accounting
        acc = ((torch.round(logits) == labels).sum().float() / bs).float()
        loss_.update(loss.mean().item(), bs)
        acc_.update(acc.item(), bs)
        time_.update(time.time() - end)

#     scheduler.step()
    return loss_, acc_, time_

def test(device, model):
    time_ = AverageMeter()
    loss_ = AverageMeter()
    acc_ = AverageMeter()
    model.eval()

    for i, data, in enumerate(train_testloader, 1):
        # Accounting
        end = time.time()

        inputs = data["x"].float()
        labels = data["y"].float()
        inputs = inputs.to(device)
        labels = labels.to(device)

        bs = inputs.size(0)

        with torch.no_grad():
            logits = model(inputs)
            loss = criterion(logits, labels)
            acc = ((torch.round(logits) == labels).sum().float() / bs).float()

            # Accounting
            loss_.update(loss.mean().item(), bs)
            acc_.update(acc.mean().item(), bs)
            time_.update(time.time() - end)

    return loss_, acc_, time_

class AverageMeter():
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_model(device)

criterion = nn.BCELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.01, nesterov=True)
optimizer = optim.Adam(model.parameters(), lr=0.01)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [25]:
num_epochs = 10

experiment(num_epochs=num_epochs,
           trainloader=train_trainloader,
           device=device,
           model=model,
           optimizer=optimizer,
           scheduler=lr_scheduler)

# 70/30 split
# Epoch 1. [Train] 	 Time 0.69 Loss                 0.71 	 Accuracy 0.50
# Epoch 1. [Test] 	 Time 0.52 T-Loss                 0.69 	 T-Accuracy 0.50
# Epoch 2. [Train] 	 Time 0.59 Loss                 0.69 	 Accuracy 0.52
# Epoch 2. [Test] 	 Time 0.52 T-Loss                 0.69 	 T-Accuracy 0.53
# Epoch 3. [Train] 	 Time 0.58 Loss                 0.67 	 Accuracy 0.59
# Epoch 3. [Test] 	 Time 0.52 T-Loss                 0.69 	 T-Accuracy 0.57
# Epoch 4. [Train] 	 Time 0.58 Loss                 0.61 	 Accuracy 0.67
# Epoch 4. [Test] 	 Time 0.52 T-Loss                 0.67 	 T-Accuracy 0.60
# Epoch 5. [Train] 	 Time 0.58 Loss                 0.49 	 Accuracy 0.77
# Epoch 5. [Test] 	 Time 0.52 T-Loss                 0.68 	 T-Accuracy 0.64
# Epoch 6. [Train] 	 Time 0.58 Loss                 0.23 	 Accuracy 0.92
# Epoch 6. [Test] 	 Time 0.51 T-Loss                 0.80 	 T-Accuracy 0.68
# Epoch 7. [Train] 	 Time 0.58 Loss                 0.11 	 Accuracy 0.96
# Epoch 7. [Test] 	 Time 0.52 T-Loss                 0.94 	 T-Accuracy 0.69
# Epoch 8. [Train] 	 Time 0.58 Loss                 0.07 	 Accuracy 0.98
# Epoch 8. [Test] 	 Time 0.52 T-Loss                 1.04 	 T-Accuracy 0.70
# Epoch 9. [Train] 	 Time 0.58 Loss                 0.04 	 Accuracy 0.99
# Epoch 9. [Test] 	 Time 0.52 T-Loss                 1.15 	 T-Accuracy 0.70
# Epoch 10. [Train] 	 Time 0.59 Loss                 0.04 	 Accuracy 0.99
# Epoch 10. [Test] 	 Time 0.52 T-Loss                 1.20 	 T-Accuracy 0.70
# Epoch 11. [Train] 	 Time 0.58 Loss                 0.03 	 Accuracy 0.99
# Epoch 11. [Test] 	 Time 0.52 T-Loss                 1.19 	 T-Accuracy 0.71

Upon initialization. [Test] 	 Time 0.00             Loss 0.69 	 Accuracy 0.50
Epoch 1. [Train] 	 Time 6.37 Loss                 0.65 	 Accuracy 0.63
Epoch 1. [Test] 	 Time 7.40 T-Loss                 0.61 	 T-Accuracy 0.66
Epoch 2. [Train] 	 Time 6.48 Loss                 0.63 	 Accuracy 0.65
Epoch 2. [Test] 	 Time 7.61 T-Loss                 0.61 	 T-Accuracy 0.67
Epoch 3. [Train] 	 Time 6.52 Loss                 0.62 	 Accuracy 0.66
Epoch 3. [Test] 	 Time 7.59 T-Loss                 0.61 	 T-Accuracy 0.67
Epoch 4. [Train] 	 Time 6.40 Loss                 0.62 	 Accuracy 0.66
Epoch 4. [Test] 	 Time 7.77 T-Loss                 0.60 	 T-Accuracy 0.68
Epoch 5. [Train] 	 Time 6.49 Loss                 0.61 	 Accuracy 0.67
Epoch 5. [Test] 	 Time 7.57 T-Loss                 0.59 	 T-Accuracy 0.68
Epoch 6. [Train] 	 Time 6.72 Loss                 0.60 	 Accuracy 0.68
Epoch 6. [Test] 	 Time 7.43 T-Loss                 0.59 	 T-Accuracy 0.68
Epoch 7. [Train] 	 Time 6.35 Loss                 0.

In [26]:
def timer(start, end):
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    return "{:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds)

predictions = np.zeros(len(test_data))
print("predicting on test_data...")
l_test = len(test_data)
start_t = time.time()

# Set model to evaluating mode
model.eval()

for i, data in enumerate(test_dataloader, 0):
    x = data["x"].float()
    
    x = x.to(device)
    predictions[i] = model(x)
    # predictions[i] = model_pred(x)
    if i % 1000 == 0:
        print(f"Predicted: {i}/{l_test}, Time elapsed: {timer(start_t, time.time())}")
        
        
# predictions_INT = pd.DataFrame(predictions).astype(int)
predictions_DF = pd.DataFrame(predictions)
predictions_rounded = predictions_DF.round(0)

# print("saving predictions...")
# predictions_INT.to_csv('data/predictions.csv', index=False, header=False)
print("done")

predicting on test_data...
Predicted: 0/59544, Time elapsed: 00:00:00.13
Predicted: 1000/59544, Time elapsed: 00:00:01.20
Predicted: 2000/59544, Time elapsed: 00:00:02.28
Predicted: 3000/59544, Time elapsed: 00:00:03.35
Predicted: 4000/59544, Time elapsed: 00:00:04.46
Predicted: 5000/59544, Time elapsed: 00:00:05.57
Predicted: 6000/59544, Time elapsed: 00:00:06.65
Predicted: 7000/59544, Time elapsed: 00:00:07.67
Predicted: 8000/59544, Time elapsed: 00:00:08.78
Predicted: 9000/59544, Time elapsed: 00:00:09.83
Predicted: 10000/59544, Time elapsed: 00:00:10.88
Predicted: 11000/59544, Time elapsed: 00:00:11.96
Predicted: 12000/59544, Time elapsed: 00:00:13.04
Predicted: 13000/59544, Time elapsed: 00:00:14.14
Predicted: 14000/59544, Time elapsed: 00:00:15.16
Predicted: 15000/59544, Time elapsed: 00:00:16.19
Predicted: 16000/59544, Time elapsed: 00:00:17.28
Predicted: 17000/59544, Time elapsed: 00:00:18.34
Predicted: 18000/59544, Time elapsed: 00:00:19.41
Predicted: 19000/59544, Time elapsed

In [27]:
# res = pd.DataFrame(predictions_rounded).astype(int)
res = pd.DataFrame(predictions_rounded).astype(int)
print("saving predictions...")
res.to_csv('data/predictions_inverse_cooking_10_epochs_batchs_10.csv', index=False, header=False)
print("done")

res.head(100)

saving predictions...
done


,0
0,1
1,0
2,1
3,0
4,1
...,...
95,1
96,1
97,1
98,0


In [11]:
Path("model").mkdir(parents=True, exist_ok=True)  # create folder if necessary
torch.save(model.state_dict(), "model/params_6_layers_mobileNetV3small_v2")
# load: model.load_state_dict(torch.load("model/params"))
